# Packages 

In [2]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

In [3]:
candidate_file_name

'{task}_{data_type}_{model_version}_{model_for_eval}_top{topn}.parquet'

In [4]:
debug = False

eval_cg = False

load_train_eval_data = False
if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'
train_dir = '../data/train_data'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model_version = 'w2v_v3'
nic_model_version = 'nic'
rank_model_version = 'rank_lgbm_v2'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100



# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")


sub_file = f'../data/sub_files/{rank_model_version}.parque'
eval_sub_file = f'../data/sub_files/eval_{rank_model_version}.parque'

In [5]:
! mkdir {rank_model_dir}

mkdir: cannot create directory ‘../model_training/rank_lgbm_v2’: File exists


In [6]:
# ! mkdir ../data/train_data

In [7]:
# model_dir

# Function 

In [8]:
! ls ../data/candidates/

task1_eval_nic_True_top100.parquet     task1_test_w2v_v3_True_top100.parquet
task1_eval_w2v_v2_top100.parquet       task1_train_nic_True_top100.parquet
task1_eval_w2v_v3_True_top100.parquet  task1_train_w2v_v2_top100.parquet
task1_test_nic_True_top100.parquet     task1_train_w2v_v3_True_top100.parquet
task1_test_w2v_v2_top100.parquet


# Original data 

In [9]:

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))


# Get candiadtes

In [10]:
# w2v_file = os.path.join(candidate_path, f'task1_{data_type}_w2v_top100.parquet')
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

In [11]:
candidate_file_name

'{task}_{data_type}_{model_version}_{model_for_eval}_top{topn}.parquet'

In [12]:
def get_all_candidates(data_type, task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version
                      ,model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn):
    
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    if data_type == 'test':
        original_file_name = f"sessions_{data_type}_{task}.parquet"
    else:
        original_file_name = f"sessions_{data_type}.parquet"
    original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    
    # get w2v weight
    w2v_pl = w2v_pl.with_columns(
        pl.lit([list(range(w2v_topn, 0, -1))]).alias('w2v_weight')
    )
    get_w2v_weight = pl.element().rank()*0
    nic_pl = nic_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                # parallel=True
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    cols = ['session_id', 'next_item_prediction', 'w2v_weight']
    combined_pl = w2v_pl.select(cols).join(nic_pl.select(cols), how='left', on='session_id', suffix='_nic')

    # combined_pl = combined_pl.with_columns(
    #     pl.concat_list([pl.col('next_item_prediction'), pl.col('next_item_prediction_nic')]).alias('next_item_prediction'), 
    #     pl.concat_list([pl.col('w2v_weight'), pl.col('w2v_weight_nic')]).alias('w2v_weight'), 
    # ).select(
    #     pl.all().exclude(['next_item_prediction_nic', 'w2v_weight_nic'])
    # )
    explode_cols = ['next_item_prediction', 'w2v_weight']
    combined_pl = (
            pl.concat([w2v_pl.select(cols).explode(explode_cols), nic_pl.select(cols).explode(explode_cols)], how='vertical')
                .groupby(['session_id', 'next_item_prediction'])
                .agg(
                    pl.col('w2v_weight').max().alias('w2v_weight')
                )
                .groupby('session_id')
                .agg(
                    pl.all()
                )
    )
    combined_pl = (
        combined_pl.join(original_pl, how='left', on='session_id')
            # .with_columns(
            #     pl.col('prev_items')
            # )
    )
    return combined_pl


In [13]:
train_cg_pl = get_all_candidates(data_type='train')
eval_cg_pl = get_all_candidates(data_type='eval')
test_cg_pl = get_all_candidates(data_type='test')

In [13]:
# eval_cg_pl.head().collect()

In [14]:
# train_cg_pl.with_columns(
#     pl.col('next_item_prediction').arr.lengths().alias('length')
#     , pl.col('next_item_prediction').arr.unique().arr.lengths().alias('unique_length')
# ).select(pl.col('length').sub(pl.col('unique_length')).alias('diff')).collect().to_series().value_counts().sort('counts',
#                                                                                                                 descending=True)

In [15]:
# train_cg_pl.collect().estimated_size(unit='gb')

In [16]:
# original_pl.collect().estimated_size(unit='gb')

In [17]:




# data_type = 'train'
# w2v_file = os.path.join(candidate_path, 
#                        candidate_file_name.format(
#                 task=task
#             , data_type=data_type
#             , model_version=w2v_model
#             , model_for_eval=model_for_eval
#             , topn=w2v_topn
#                        ))
# nic_file = os.path.join(candidate_path,
#             candidate_file_name.format(
#                 task=task
#                 , data_type=data_type
#                 , model_version=nic_model
#                 , model_for_eval=model_for_eval
#                 , topn=nic_topn
#                        ))
# if data_type == 'test':
#     original_file_name = f"sessions_{data_type}_{task}.parquet"
# else:
#     original_file_name = f"sessions_{data_type}.parquet"
# original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

# # get w2v weight
# w2v_pl = w2v_pl.with_columns(
#     pl.lit([list(range(w2v_topn, 0, -1))]).alias('w2v_weight').cast(pl.List(pl.Int8))
# ).collect()


In [18]:
# get_w2v_weight = pl.element().rank()*0
# nic_pl = nic_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                             # parallel=True
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int8))
# ).collect()

In [19]:
# train_cg_pl.collect()

In [20]:
# eval_cg_pl.collect().estimated_size(unit='gb')

In [21]:
# train_cg_pl.collect()

In [22]:
# print(f"{train_cg_pl.collect().shape}; {eval_cg_pl.collect().shape}; {test_cg_pl.collect().shape}")

In [23]:
# train_cg_pl.collect().head()

In [24]:
# eval_cg_pl.collect()

In [25]:
# get_w2v_weight = pl.element().rank()*0

# train_pl.with_columns(
#     # pl.lit([list(range(100, 0, -1))]).alias('w2v_weight')
#     pl.col('next_item_prediction').arr.lengths().alias('length')
#     , pl.col('next_item_prediction').arr.eval(get_w2v_weight, parallel=True).alias('w2v_weight')
#     # , pl.lit([0 for ele in range(pl.col('length'))])

# ).head().collect()

In [26]:
# dir(pl.element())

In [27]:
# w2v_pl.head().collect()


In [28]:

# def get_all_candidates(data_type):
#     w2v_file = os.path.join(candidate_path, f'task1_{data_type}_w2v_top100.parquet')
#     nic_file = os.path.join(candidate_path, f'task1_{data_type}_nic_top100.parquet')
#     w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
#     nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

#     cols_to_keep = ['session_id', 'next_item_prediction', 'cg_source']

#     w2v_pl = w2v_pl.with_columns(pl.lit('word2vec').alias('cg_source')).select(cols_to_keep).explode('next_item_prediction')
#     nic_pl = nic_pl.with_columns(pl.lit('next_item_counter').alias('cg_source')).select(cols_to_keep).explode('next_item_prediction')
#     # length = w2v_pl.collect().shape[0]
#     # w2v_pl = w2v_pl.with_columns(
#     #     pl.Series(values=[list(range(100, 0, -1))]*length, name='w2v_weight')
#     # )
#     cg_pl = (
#             pl.concat([w2v_pl, nic_pl], how='vertical')
#                 .groupby(['session_id', 'next_item_prediction'])
#                 .agg(
#                     pl.col('cg_source')
#                 )
#                 .with_columns(
#                     pl.when(pl.col('cg_source').arr.contains(pl.lit('word2vec'))).then(1).otherwise(0).alias('whether_w2v')
#                     , pl.when(pl.col('cg_source').arr.contains(pl.lit('next_item_counter'))).then(1).otherwise(0).alias('whether_nic')
#                 )
#                 .select(
#                     pl.all().exclude('cg_source')
#                 )
#     )
#     return cg_pl

# Eval candidate generation 

In [29]:
if eval_cg:
    eval_topn = 300

    col = f"recall@{eval_topn}"

    eval_final = (
            eval_cg_pl
            .lazy()
            .with_columns(
                pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
            )
            .with_columns(
                pl.concat_list([pl.col('next_item'), pl.col('next_item_prediction')]).alias('mrr')
            )
            ).with_columns(
                pl.col('next_item_prediction').arr.head(eval_topn).arr.contains(pl.col('next_item')).mean().alias(col)

            )
    final_res = eval_final.select(
            pl.count().alias('total_sessions')
            , pl.col(col).mean()

        ).collect()
    print(final_res)
    del final_res

In [30]:
# eval_cg_pl.head().collect()

# Feature Process 

## Load NIC, W2V Model 

In [14]:
nic_model = (
    pl.scan_parquet('../model_training/next_item_counter_v2/nic_model.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
        )
            )
print(nic_model.schema)



# w2v_model_file = '../model_training/v2/w2v.model'
w2vec = Word2Vec.load(w2v_model_file)
annoy_index = AnnoyIndexer()
annoy_index.load(annoy_index_file)

{'next_item_prediction': Utf8, 'next_item_weight': Float64, 'last_prev_item': Utf8}


In [15]:
# nic_model

In [16]:
# len(w2vec.wv)

In [17]:
# train_cg_pl.schema

In [18]:
# w2vec.wv.similarity('2', '3')

In [19]:
# train_cg_pl = train_cg_pl.collect()

In [20]:
# train_cg_pl.shape

In [21]:
# train_cg_pl.head()

In [22]:
# target_df.head().collect()

In [23]:
# pos.head()/collect()

In [24]:
# def get_w2v_simi(x):
#     try:
#         simi = w2vec.wv.similarity(x['next_item_prediction'],
#                                                   x['last_prev_item']
#                                                  )
#     except:
#         simi = 0
#     return simi

In [25]:
# target_df.head().collect()

In [26]:
# target_df = target_df.with_columns(
#                 pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
#                 , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
#                     lambda x: get_w2v_simi(x)).alias('last_item_similarity')
#                 , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
#                     .then(2)
#                     .otherwise(3).alias('locale')
#             ).sort('session_id')

## get features 

In [27]:
# target_df = train_candidates
# data_type = 'train'

def get_w2v_simi(x, col='last_prev_item'):
    try:
        if isinstance(x[col], str):
            target = [x[col]]
        else:
            target = x[col]
        simi = w2vec.wv.n_similarity([x['next_item_prediction']],
                                                  target
                                                 )
    except Exception as e:
        print(e)
        # print(x[col])
        simi = 0
    return simi

def get_feature(target_df, data_type, nic_model=nic_model, negative_sampling=None):

    target_df = target_df.with_columns(
                    pl.col('prev_items').arr.lengths().alias('prev_length')
                                      ).explode(['next_item_prediction', 'w2v_weight'])

    if data_type != 'test':
        target_df = (
            target_df
                .with_columns(
                    pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
                )
        )

    if negative_sampling is not None:
        target_df = target_df.collect()
        pos = target_df.filter(pl.col('target')==1)
        neg = target_df.filter(pl.col('target')==0).sample(frac=negative_sampling)
        target_df = pl.concat([pos, neg], how='vertical')
        print(target_df.select('target').mean())

    target_df = (
            target_df.lazy().with_columns(
                    pl.col('prev_items').arr.get(-1).alias('last_prev_item')
                ).join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
                .with_columns(
                    pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
                    , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
                        lambda x: get_w2v_simi(x, col='last_prev_item')).alias('last_item_similarity').cast(pl.Float32)
                    , pl.struct(["next_item_prediction", "prev_items"]).apply(
                        lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
                    , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                        .then(2)
                        .otherwise(3).alias('locale')
                ).sort('session_id')

    )
    return target_df
# target_df.head(3).collect()

In [28]:
if not load_train_eval_data:
    sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', negative_sampling=0.1)
    sampled_eval_candidates = get_feature(target_df=eval_cg_pl, data_type='eval', negative_sampling=0.1)
else:
    sampled_train_candidates = pl.read_parquet(os.path.join(train_dir, 'train.parquet'), n_rows=SAMPLE_NUM)
    sampled_eval_candidates = pl.read_parquet(os.path.join(train_dir, 'eval.parquet'), n_rows=SAMPLE_NUM)

shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.078419 │
└──────────┘
shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.041159 │
└──────────┘


In [ ]:
# sampled_train_candidates.head().collect()

In [ ]:
# eval_candidates.collect()

In [ ]:
# target_df = eval_cg_pl
# data_type = 'eval'
# (
#     target_df.explode(['next_item_prediction', 'w2v_weight'])
#     .with_column(
#             pl.col('prev_items').arr.get(-1).alias('last_prev_item')
#         )
#         .join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
#         .with_columns(
#             pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
#             # , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
#             #     lambda x: w2vec.wv.similarity(x['next_item_prediction'],
#             #                                   x['last_prev_item']
#             #                                  )).alias('last_item_similarity')
#             , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
#                 .then(2)
#                 .otherwise(3).alias('locale')
#         ).sort('session_id')
# ).head().collect()
# # if data_type != 'test':
# #     target_df = (
# #         target_df
# #             .with_columns(
# #                 pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
# #             )
# #     )
# # target_df.head().collect()

In [ ]:
# eval_candidates.head().collect()

In [ ]:
# test_candidates.collect()

In [ ]:
# test_candidates.head().collect()

# Ranker 

## Config 

In [28]:
estimator = 1000
early_stop_rounds = 10
if debug:
    early_stop_rounds = 5
    
target = 'target'
feature_cols = [
    # 'whether_w2v'
    # , 'whether_nic'
    'next_item_weight'
    , 'locale'
    , 'w2v_weight'
    , 'last_item_similarity'
    , 'prev_item_similarity'	
    , 'prev_length'
]

categorical_feature=['locale']

## Train model

In [ ]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    n_estimators=estimator, 
    importance_type='gain',
    eval_at=[3]
)

In [ ]:
if not load_train_eval_data:
    sampled_train_candidates = sampled_train_candidates.collect()
    sampled_train_candidates.shape
    sampled_eval_candidates = sampled_eval_candidates.collect()
    sampled_eval_candidates.shape
if not debug and not load_train_eval_data:
    sampled_train_candidates.write_parquet(os.path.join(train_dir, 'train.parquet'))
    sampled_eval_candidates.write_parquet(os.path.join(train_dir, 'eval.parquet'))

In [ ]:
# sampled_train_candidates = pl.read_parquet(os.path.join(train_dir, 'train.parquet'))
# sampled_eval_candidates = pl.read_parquet(os.path.join(train_dir, 'eval.parquet'))

In [ ]:
print(sampled_train_candidates.select('target').mean())


In [ ]:
print(sampled_eval_candidates.select('target').mean())

In [ ]:
session_lengths_train = sampled_train_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()
session_lengths_eval = sampled_eval_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()

In [ ]:
ranker = ranker.fit(
    X=sampled_train_candidates[feature_cols].to_pandas(),
    y=sampled_train_candidates[target].to_pandas(),
    group=session_lengths_train,
    eval_set=[(sampled_train_candidates[feature_cols].to_pandas(), sampled_train_candidates[target].to_pandas()),
           (sampled_eval_candidates[feature_cols].to_pandas(), sampled_eval_candidates[target].to_pandas())
             ],
    eval_group=[session_lengths_train,
                session_lengths_eval
               ]
    , categorical_feature=categorical_feature
    , callbacks=[early_stopping(early_stop_rounds)]
)

In [ ]:
print(sampled_train_candidates.estimated_size(unit='gb'))
del sampled_train_candidates

In [ ]:
print(sampled_eval_candidates.estimated_size(unit='gb'))
del sampled_eval_candidates

## Save model 

In [ ]:
rank_model_dir

In [ ]:
if not debug:
    joblib.dump(
            value=ranker,
            filename=os.path.join(rank_model_dir, 'model.pkl')
    )

In [ ]:
if not debug:
    del ranker

## Load Model 

In [29]:
if not debug:
    ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))
ranker

LGBMRanker(eval_at=[3], importance_type='gain', metric='ndcg',
           n_estimators=1000, objective='lambdarank')

In [30]:
# ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))


### Importance 

In [31]:
impotant_df = pd.DataFrame(
    {
        'features': ranker.feature_name_,
        'importance': ranker.feature_importances_
    }
).sort_values('importance', ascending=False)
impotant_df

,features,importance
0,next_item_weight,1.259315e+07
3,last_item_similarity,1.358875e+05
4,prev_item_similarity,1.982588e+04
2,w2v_weight,1.630157e+04
5,prev_length,9.391630e+02
1,locale,0.000000e+00


# Eval ranker 

In [32]:
def rank_rec(candidate_pl, feature_cols, ranker, topn=100):

    candidate_pl = candidate_pl.collect()
    print(f"candidate pl shape: {candidate_pl.shape}")
    inference = ranker.predict(candidate_pl[feature_cols].to_pandas())
    test_result = (candidate_pl
         .lazy()
         .with_columns(
             pl.Series(name='predict', values=inference)
         )
         .with_columns(
            pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
         )
         .sort(['session_id', 'rank'])
         .filter(pl.col('rank')<=topn)
         .groupby(['session_id'])
         .agg(
             pl.col('next_item_prediction')
             , pl.col('next_item').unique()#.arr.get(0).cast(pl.Utf8)
         ).with_columns(
             pl.col('next_item').arr.get(0).alias('next_item')
         )
    )
    return test_result


In [33]:
# if load_train_eval_data:
#     pass
# else:
eval_candidates = get_feature(target_df=eval_cg_pl, data_type='eval', 
                              # negative_sampling=0.1
                             )

In [ ]:
ranker_eval_pl = rank_rec(candidate_pl=eval_candidates,
                         feature_cols=feature_cols
                         , ranker=ranker
                        # , topn=10
                         )

In [ ]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

In [ ]:
eval_sub_file

In [ ]:
ranker_eval_pl.collect().write_parquet(eval_sub_file)

# Test inference 

In [41]:
test_cg_pl.schema

{'session_id': Int64,
 'next_item_prediction': List(Utf8),
 'w2v_weight': List(Int64),
 'prev_items': Unknown,
 'locale': Utf8}

In [42]:
# nic_model

In [43]:
test_candidates = get_feature(target_df=test_cg_pl, data_type='test')

In [44]:
test_candidates = test_candidates.collect()
test_candidates.shape

(124278, 10)

In [45]:
inference = ranker.predict(test_candidates[feature_cols].to_pandas())

In [46]:
inference

array([-0.2, -0.2, -0.2, ..., -0.2, -0.2, -0.2])

In [47]:
test_result = (test_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=100)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test_result.head().collect()

In [48]:
test_pl.schema

{'prev_items': Unknown, 'locale': Utf8, 'session_id': Int64}

In [49]:
predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [50]:
predictions.shape

(1000, 2)

In [51]:
predictions.shape

(1000, 2)

In [ ]:
# train_candidates.estimate_size(unit='gb')

In [ ]:
# original_test.head()

In [ ]:
# ! ls -al ../data/sub_files/

In [ ]:
sub_file

In [ ]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet(sub_file, engine='pyarrow')

In [ ]:
# You can submit with aicrowd-cli, or upload manually on the challenge page.
!aicrowd submission create -c task-1-next-product-recommendation -f {sub_file}

# submission file check 

In [ ]:
# sub_df = pl.scan_parquet(sub_file)
# test_df = pl.scan_csv('../data/raw_data/sessions_test_task1.csv')

In [ ]:
# sub_df.with_columns(
#     pl.col('next_item_prediction').arr.lengths().alias('length')
#     , pl.col('next_item_prediction').arr.unique().arr.lengths().alias('unique_length')
# ).select('unique_length').collect().to_series().